# Set Membership

The cell below defines two **abstract classes**: the first represents a set and basic insert/search operations on it. You will need to impement this API four times, to implement (1) sequential search, (2) binary search tree, (3) balanced search tree, and (4) bloom filter. The second defines the synthetic data generator you will need to implement as part of your experimental framework. <br><br>**Do NOT modify the next cell** - use the dedicated cells further below for your implementation instead. <br>

In [2]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

# abstract class to represent a set and its insert/search operations
class AbstractSet(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # inserts "element" in the set
    # returns "True" after successful insertion, "False" if the element is already in the set
    # element : str
    # inserted : bool
    @abstractmethod
    def insertElement(self, element):     
        inserted = False
        return inserted   
    
    # checks whether "element" is in the set
    # returns "True" if it is, "False" otherwise
    # element : str
    # found : bool
    @abstractmethod
    def searchElement(self, element):
        found = False
        return found    
    
    
    
# abstract class to represent a synthetic data generator
class AbstractTestDataGenerator(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # creates and returns a list of length "size" of strings
    # size : int
    # data : list<str>
    @abstractmethod
    def generateData(self, size):     
        data = [""]*size
        return data   


Use the cell below to define any auxiliary data structure and python function you may need. Leave the implementation of the main API to the next code cells instead.

In [3]:
# ADD AUXILIARY DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE

class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

class RB_Node(Node):
    RED = True
    BLACK = False

    def __init__(self, value, colour):
        super().__init__(value)
        self.colour = colour

Use the cell below to implement the requested API by means of **sequential search**.

In [4]:
class SequentialSearchSet(AbstractSet):
    def __init__(self):
        self.set = []      
     
    def insertElement(self, element):
        if self.searchElement(element) is False:
            self.set.append(element)
            return True
        return False
    
    def searchElement(self, element):
        for member in self.set:
            if member == element:
                return True
        return False

Use the cell below to implement the requested API by means of **binary search tree**.

In [5]:
class BinarySearchTreeSet(AbstractSet):
    def __init__(self):
        self.root = None
    
    def insertElement(self, element):
        if not self.root:
            self.root = Node(element)
            return True
        else:
            current = self.root
            while current:
                if current.value == element:
                    return False
                elif current.value > element:
                    if not current.left:
                        current.left = Node(element)
                        return True
                    current = current.left
                elif current.value < element:
                    if not current.right:
                        current.right = Node(element)
                        return True
                    current = current.right
        return False

    def searchElement(self, element):
        current = self.root
        while current:
            if current.value == element:
                return True
            elif current.value > element:
                current = current.left
            elif current.value < element:
                current = current.right
        return False


Use the cell below to implement the requested API by means of **balanced search tree**.

In [6]:
class BalancedSearchTreeSet(AbstractSet):
    def __init__(self):
        self.root = None

    @staticmethod
    def is_red(node):
        if node is None:
            return False
        return node.colour == RB_Node.RED

    @staticmethod
    def rotate_left(node):
        x = node.right
        node.right = x.left
        x.left = node
        x.colour = node.colour
        node.colour = RB_Node.RED
        return x

    @staticmethod
    def rotate_right(node):
        x = node.left
        node.left = x.right
        x.right = node
        x.colour = node.colour
        node.colour = RB_Node.RED
        return x

    @staticmethod
    def flip_colours(node):
        node.colour = RB_Node.RED
        node.left.colour = RB_Node.BLACK
        node.right.colour = RB_Node.BLACK
    
    def insertElement(self, element):
        self.root, inserted = self._insertElement(self.root, element)
        self.root.colour = RB_Node.BLACK
        return inserted
    
    def _insertElement(self, node, element):
        if node is None:
            return RB_Node(element, RB_Node.RED), True

        if element < node.value:
            node.left, inserted = self._insertElement(node.left, element)
        elif element > node.value:
            node.right, inserted = self._insertElement(node.right, element)
        else:
            return node, False
        
        if self.is_red(node.right) and not self.is_red(node.left):
            node = self.rotate_left(node)
        if self.is_red(node.left) and self.is_red(node.left.left):
            node = self.rotate_right(node)
        if self.is_red(node.left) and self.is_red(node.right):
            self.flip_colours(node)
        return node, inserted    

    def searchElement(self, element):     
        return self._searchElement(self.root, element) == element

    def _searchElement(self, node, element):
        if node is None:
            return False

        if element < node.value:
            return self._searchElement(node.left, element)
        elif element > node.value:
            return self._searchElement(node.right, element)
        else:
            return node.value   

Use the cell below to implement the requested API by means of **bloom filter**.

In [7]:
from bitarray import bitarray

EXPECTED_NUM_INSERTIONS = 100000
BITARRAY_SIZE = EXPECTED_NUM_INSERTIONS * 7
LN2 = 0.69314718056

class BloomFilterSet(AbstractSet):
    def __init__(self, size, num_hashes=3):
        self.size = size
        self.num_hashes = num_hashes
        self.bit_array = bitarray(size)
        self.bit_array.setall(0)

    def _hash(self, element):
        hash_values = []
        for i in range(self.num_hashes):
            hash_value = hash(f'{element}{i}')
            hash_values.append(hash_value % self.size)
        return hash_values
    
    @staticmethod
    def optimise_k(m, n):
        return 1 + int(LN2 * (m / n))
        
    def insertElement(self, element):
        if self.searchElement(element) is False:
            hash_values = self._hash(element)
            for value in hash_values:
                self.bit_array[value] = 1
            return True
        else:
            return False
    
    def searchElement(self, element):     
        hash_values = self._hash(element)
        for value in hash_values:
            if not self.bit_array[value]:
                return False
        return True

Use the cell below to implement the **synthetic data generator** as part of your experimental framework.

In [8]:
import string
import random

class TestDataGenerator(AbstractTestDataGenerator):
    NUM_WORDS = 50000
    MIN_LENGTH = 1
    AV_LOW_LENGTH = 2
    AV_HIGH_LENGTH = 6
    MAX_LENGTH = 12
    CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
    VOWELS = 'aeiou'
    test_file = "synthetic_test_search.txt"
    SAMPLE_INTERVAL = 200
    test_words = set()

    def __init__(self):
        pass

    def generate_word(self, scarcity=0.05):
        limit = int(1 / scarcity)
        roulette = random.randint(0, int(1 / scarcity))
        if roulette == limit:
            length = random.randint(self.AV_LOW_LENGTH, self.AV_HIGH_LENGTH)
        else:
            length = random.randint(self.MIN_LENGTH, self.MAX_LENGTH)
        word = ''
        for i in range(length):
            if i % 2 == 1 or length == 1:
                word += random.choice(self.VOWELS)
            else:
                word += random.choice(self.CONSONANTS)
        return word

    # Generate the words and put them in an array + create a test file
    def generateData(self, size):
        data = []
        for i in range(size):
            word = self.generate_word()
            data.append(word)
            if i % self.SAMPLE_INTERVAL == 0:
                self.test_words.add(word)
                self.test_words.add(self.generate_word())

        test_words = list(self.test_words)
        with open(self.test_file, 'w') as f:
            for i in range(len(test_words)):
                f.write(test_words[i])
                if i <= len(test_words):
                    f.write('\n')

        return data


Use the cells below for the python code needed to **fully evaluate your implementations**, first on real data and subsequently on synthetic data (i.e., read data from test files / generate synthetic one, instantiate each of the 4 set implementations in turn, then thorouhgly experiment with insert/search operations and measure their performance).

In [9]:
# ADD YOUR TEST CODE HERE TO WORK ON REAL DATA
import timeit
import random
import string

class ExperimentalFramework:

    def __init__(self, real_data_file, data_structures_dict, search_test_file="test-search.txt", operation_repeats=50, search_intervals=5):
        self.real_data_file = real_data_file
        self.search_test_file = search_test_file
        # data_structures_dict refers to the dictionary containing the data structures you would like to test
        self.data_structures_dict = data_structures_dict
        self.operation_repeats = operation_repeats
        # Search intervals refer to the interval of items after which the search function for the data structure will be executed
        self.search_intervals = search_intervals

    def txt_to_list(self, file_path):
        with open(file_path, 'r') as f:
            content = f.read()
            if file_path == "test-search.txt":
                return content.split("\n")
            else:
                return content.split()

    def calculate_insert_and_mean_search_time(self):
        real_data_values = self.txt_to_list(self.real_data_file)
        search_words = self.txt_to_list(self.search_test_file)

        insert_time = {}
        mean_search_time = {}
        intervals = len(real_data_values) // self.search_intervals

        for data_structure_name, data_structure in self.data_structures_dict.items():
            print(data_structure_name)
            insert_time[data_structure_name] = {}
            mean_search_time[data_structure_name] = {}

            # Iterate through the real_data_values and time insert and search times
            for i, word in enumerate(real_data_values):
                insert_time[data_structure_name][i] = timeit.timeit(lambda: data_structure.insertElement(word), number=self.operation_repeats)

                if i % intervals == 0:
                    # Executes the search function in specified intervals for the data structure over the elements already input
                    search_time = [timeit.timeit(lambda: data_structure.searchElement(word), number=self.operation_repeats) for word in search_words]
                    # calculates the mean search time over this set and stores it
                    mean_search_time[data_structure_name][i] = sum(search_time) / (self.operation_repeats * len(search_time))
            print(f"Mean Search Time - {mean_search_time}")
        return insert_time, mean_search_time

    def get_insert_and_mean_search_time(self):
        insert_time, mean_search_time = self.calculate_insert_and_mean_search_time()
        
    def run_tests(self):
        


# Now for testing purposes:
data_structures_dict = {"BST": BinarySearchTreeSet()}
test1 = ExperimentalFramework("test1-mobydick.txt", data_structures_dict, "test-search.txt", 10, 100)
test1.get_insert_and_mean_search_time()

BST
Mean Search Time - {'BST': {0: 2.664402276790197e-07, 2093: 8.802347573015942e-07, 4186: 9.24416240988248e-07, 6279: 1.0201838529677293e-06, 8372: 1.0417653311795871e-06, 10465: 9.501897388163509e-07, 12558: 1.1233497761326645e-06, 14651: 1.119566374296032e-06, 16744: 1.107701100409031e-06, 18837: 1.0825505740769687e-06, 20930: 1.0847023671168253e-06, 23023: 9.816189065849015e-07, 25116: 1.0930907105121317e-06, 27209: 1.1181247253996244e-06, 29302: 1.0851549770717228e-06, 31395: 9.88416151579367e-07, 33488: 9.91888307974836e-07, 35581: 1.1202553073542381e-06, 37674: 1.1473297090226904e-06, 39767: 1.132974097795716e-06, 41860: 1.0385397830209055e-06, 43953: 1.0077063626515755e-06, 46046: 1.113925528618584e-06, 48139: 1.1411648021515357e-06, 50232: 1.0815102376317213e-06, 52325: 9.998613584000583e-07, 54418: 1.0001453049822685e-06, 56511: 1.0002845013941373e-06, 58604: 1.010960715592181e-06, 60697: 1.0047557362175863e-06, 62790: 1.0029189801270808e-06, 64883: 1.0040687469748455e-06, 

In [ ]:
import timeit

# ADD YOUR TEST CODE HERE TO WORK ON SYNTHETIC DATA



